In [ ]:
import numpy as np
import pandas as pd
import json
import panel as pn
from matplotlib import pyplot as plt
import holoviews as hv
import hvplot.pandas
from PIL import Image
import os, os.path
from bokeh.models import HoverTool, ColorBar, LinearColorMapper, LogColorMapper, Label
from bokeh.plotting import figure
from bokeh.layouts import row
import os

In [ ]:
os.chdir('/Users/chang61/Documents/PCMDI/peak_season_cmec')

In [ ]:
# load data from json file
f = 'data/peak_season_cmec.json'
with open(f) as og_file:
    dat = json.load(og_file)

In [ ]:
dat

In [ ]:
# convert json data to pandas dataframe 
ps_df = pd.DataFrame.from_dict(dat['RESULTS'], orient='index')

# format pandas df for portrait plot
ps_df['Model'] = ps_df.index
ps_df = ps_df.reset_index()
ps_df = ps_df[['Model', 'peak season']]
ps_df['var_name'] = 'peak season'
ps_df['region_name'] = 'region 1'
ps_df

In [ ]:
# load histogram png files
imgs = []
path = "./images/"
for p in os.listdir(path):
    ext = os.path.splitext(p)[1]
    imgs.append(Image.open(os.path.join(path,p)))

# view one image to confirm
#imgs[0].show()

In [ ]:
# add image links to df
# edit hyperlinks from github repo to match format below (add 'raw.githubusercontent.com', remove 'blob' from path)
# link order should match order of models displayed in df above

img_links = ['https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/BCC-CSM2-MR.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/CSIRO-Mk3.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/CanESM2.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/ERA5.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/MERRA2.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/MRI-ESM2.png',
             'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/NorESM1.png', 
             ]

ps_df['img'] = img_links

# adjust pandas settings to view full column width
pd.set_option('max_colwidth', 1000)

ps_df

In [ ]:
# html for custom hover tool
hover = HoverTool(tooltips="""
    <div>
        <div>
            <img src="@img" width=400 style="float: left; margin: 0px 15px 15px 0px; border="2"></img>
        </div>
    </div>

""")

In [ ]:
peak_plot01 = ps_df.hvplot.heatmap(y='var_name',
                       x='Model',
                       C='peak season',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = 300,
                       width=950,
                       colorbar=True,
                       xaxis='top',
                       cmap='bwr').opts(xrotation=45, fontsize={
                           'labels': 14,
                           'xticks': 14,
                           'yticks': 14
                       })
peak_plot01

In [ ]:
# save portrait plot to charts folder as an html file
# change file name each time saving a new version
# update index.md file to match new html file name

#hvplot.save(peak_plot01, 'charts/peak_plot10.html')

# push changes to github to see updates on live webpage

In [28]:
# creating reanalyses portrait plot
peak = np.loadtxt('data/output_peak_reanalysis_full.txt')
angle = peak.reshape(3,-1)

In [29]:
angle = np.where(angle < -182, angle+365, angle)
angle = np.where(angle >  182, angle-365, angle)

print(angle)

[[   7.   63.  -22. -102.  145.  155.    5.   56.  -68. -106.  138.  165.]
 [   5.   75.  -40. -102.  166.  151.  -56.   -9.   14.  -91.  -89.   14.]
 [ -49.  -11.   15.  -92.  -92.   15.  -64.  -13.   10.  -63.  -61.  -52.]]


In [30]:
model_names = ['ERA5','MERRA2','JRA55C']
region_names = ['California','SAmerica', 'Africa','NEurope','Australia','SAfrica', 'Baja', 'PAC NW', 'New Zealand', 'Alaska', 'UK', 'WEurope']

da = pd.DataFrame(data=angle, index=model_names, columns=region_names)

dd = da.stack()
dd = dd.reset_index()
dd = dd.rename(columns={"level_0": "model", "level_1": "region", 0 :"peak"})
ddd = dd['peak'].values

#print(dd)

img_path = 'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/'
img_links = []

for i, model in enumerate(model_names):
    for j, region in enumerate(region_names):
        filename = img_path+'fig_re_'+str(i)+"_"+str(j)+'.png'
        img_links.append(filename)

In [31]:
dd['img'] = img_links

#print(dd)

# adjust pandas settings to view full column width
pd.set_option('max_colwidth', 1000)

In [32]:
dd

,model,region,peak,img
0,ERA5,California,7.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_0.png
1,ERA5,SAmerica,63.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_1.png
2,ERA5,Africa,-22.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_2.png
3,ERA5,NEurope,-102.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_3.png
4,ERA5,Australia,145.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_4.png
5,ERA5,SAfrica,155.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_5.png
6,ERA5,Baja,5.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_6.png
7,ERA5,PAC NW,56.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_7.png
8,ERA5,New Zealand,-68.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_8.png
9,ERA5,Alaska,-106.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_re_0_9.png


In [39]:
hover = HoverTool(tooltips="""
    <div>
        <div>
            <img src="@img" width=400 style="float: left; margin: 0px 15px 15px 0px; border="2"></img>
        </div>
    </div>

""")
# Use number of models and regions to determine cell size
num_models = len(dd['model'].unique())
num_regions = len(dd['region'].unique())

cell_width = 1/num_models
cell_height = 1/num_regions

# set desired figure size
desired_width = 1300
desired_height = 10000

# calculate final width and height based on above parameters
adjusted_width = round(int(cell_width * desired_width), -1)
adjusted_height = round(int(cell_height * desired_height), -1)


reanalyses_plot = dd.hvplot.heatmap(y='region',
                       x='model',
                       C='peak',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = adjusted_height,
                       width = adjusted_width,
                       colorbar=True,
                       clabel='peak day',
                       xaxis='top',
                       clim = (-180,180),
                       cmap='RdBu_r'
                       ).opts(xrotation=45, 
                              fontsize={
                                  'labels':10,
                                  'xticks': 10, 
                                  'yticks': 10
                                  }
                              )

reanalyses_plot = reanalyses_plot * hv.Labels(reanalyses_plot)
reanalyses_plot 

:Overlay
   .HeatMap.I :HeatMap   [model,region]   (peak,img)
   .Labels.I  :Labels   [model,region]   (peak,img)

In [40]:
hvplot.save(reanalyses_plot , 'charts/reanalyses_plot02.html')

In [43]:
# creating portrait plot for Bias vs. ERA5 plot
# load data as array and create df
peak = np.loadtxt('data/output_diff_all_region_full.txt')
angle = peak.reshape(5,-1)

angle = np.where(angle < -182, angle+365, angle)
angle = np.where(angle >  182, angle-365, angle)

model_names = ["cmip5_CanESM2","cmip5_CCSM4","cmip5_CSIRO-Mk3-6-0","cmip5_NorESM1-M","cmip6_MRI-ESM2-0"]
region_names = ['California','SAmerica', 'Africa','NEurope','Australia','SAfrica', 'Baja', 'PAC NW', 'New Zealand', 'Alaska', 'UK', 'WEurope']

da = pd.DataFrame(data=angle, index=model_names, columns=region_names)

dd = da.stack()
dd = dd.reset_index()
dd = dd.rename(columns={"level_0": "model", "level_1": "region", 0 :"peak"})

img_path = 'https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/'
img_links = []

for i, model in enumerate(model_names):
    for j, region in enumerate(region_names):
        filename = img_path+'fig_'+str(i)+"_"+str(j)+'.png'
        img_links.append(filename)

dd['img'] = img_links
dd

,model,region,peak,img
0,cmip5_CanESM2,California,-6.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_0.png
1,cmip5_CanESM2,SAmerica,20.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_1.png
2,cmip5_CanESM2,Africa,-148.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_2.png
3,cmip5_CanESM2,NEurope,-3.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_3.png
4,cmip5_CanESM2,Australia,-16.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_4.png
5,cmip5_CanESM2,SAfrica,-81.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_5.png
6,cmip5_CanESM2,Baja,44.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_6.png
7,cmip5_CanESM2,PAC NW,-5.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_7.png
8,cmip5_CanESM2,New Zealand,8.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_8.png
9,cmip5_CanESM2,Alaska,-39.0,https://raw.githubusercontent.com/kristinchang3/peak_season_cmec/main/images/fig_0_9.png


In [50]:
# html for custom hover tool
hover = HoverTool(tooltips="""
    <div>
        <div>
            <img src="@img" width=400 style="float: left; margin: 0px 15px 15px 0px; border="2"></img>
        </div>
    </div>

""")

# Use number of models and regions to determine cell size
num_models = len(dd['model'].unique())
num_regions = len(dd['region'].unique())

cell_width = 1/num_models
cell_height = 1/num_regions

# set desired figure size
desired_width = 6000
desired_height = 6000

# calculate final width and height based on above parameters
adjusted_width = round(int(cell_width * desired_width), -1)
adjusted_height = round(int(cell_height * desired_height), -1)

bias_vs_era5 = dd.hvplot.heatmap(y='model',
                       x='region',
                       C='peak',
                       hover_cols = ['img'],
                       tools = [hover],
                       height = adjusted_height,
                       width=adjusted_width,
                       colorbar=True,
                       clabel = 'peak day',
                       xaxis='top',
                       clim = (-180,180),
                       cmap='RdBu_r').opts(xrotation=45, fontsize={
                           'labels': 14,
                           'xticks': 14,
                           'yticks': 14
                       })
bias_vs_era5 = bias_vs_era5 * hv.Labels(bias_vs_era5)
bias_vs_era5

:Overlay
   .HeatMap.I :HeatMap   [region,model]   (peak,img)
   .Labels.I  :Labels   [region,model]   (peak,img)

In [51]:
# save portrait plot to charts folder as an html file
# change file name each time saving a new version
# update index.md file to match new html file name

hvplot.save(bias_vs_era5, 'charts/bias_vs_era5.html')

# push changes to github to see updates on live webpage